# Model-Space Orthogonalization for Predictive Bayesian Model Combination

The following notebook outlines an implementation of the model orthogonalization and Bayesian model combination procedure detailed in \<Waiting for link :D\>

The layout is as follows:

- [Preamble](#preamble)
- [Setting up the Models](#setting-up-the-models)
    - [Model Parameters and Dataset Variables](#model-parameters-and-data-variables)
    - [Dataset Preparation](#dataset-preparation)



## Preamble

Up first we have a ton of preamble content that is likely only interesting to a select few, you can safely skip this part if you're just curious about the model orthogonalization and model combination aspects

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import corner 



#Some colors that Pablo likes:
colors = [
    "#ff7f0e",

    "#1f77b4",

    "#2ca02c",
    "#d62728",
    
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
] 

markers = [
    "o",  # Circle
    "^",  # Triangle up
    "s",  # Square
    "P",  # Plus (filled)
    "*",  # Star
    "X",  # X (filled)
    "D",  # Diamond
    "H",  # Hexagon
]



The following sets up some helper functions for plots that will come later on

In [ ]:
def plot_bars(values, labels, title="Bar Plot", color='blue'):
    """
    Create a bar plot based on the provided values and labels.

    Parameters:
    - values (list): A list of numerical values for the bars.
    - labels (list): A list of labels for each bar.
    - title (str): The title for the bar plot. Default is "Bar Plot".
    - color (str): Color for the bars. Default is 'blue'.
    """
    
    if len(values) != len(labels):
        raise ValueError("Length of values and labels should be the same.")
    
    plt.figure(figsize=(10, 5))
    plt.bar(labels, values, color=color)
    plt.title(title)
    plt.xlabel('Models')
    plt.ylabel('Vt Coordinates')
    plt.xticks(labels,fontsize=10,rotation='vertical')
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()


def Plotter2D_single(data):
    xvals,yvals,zvals = data
    #A plotter to see principal components in 2D
    plt.rc("xtick", labelsize=25)
    plt.rc("ytick", labelsize=25)
    
    fig, ax = plt.subplots(figsize=(12, 8), dpi=200)

    # Create scatter plot
    sc = ax.scatter(xvals, yvals, c=zvals, s=25, cmap='plasma',marker='s')
    # plt.colorbar(sc, label='Z-Value')
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label('Z-Value', fontsize=25) 
    cbar.ax.tick_params(labelsize=20) 
    plt.xlabel('Neutrons',fontsize=25)
    plt.ylabel('Protons',fontsize=25)

    ax.grid(True)
    # ax.axis('equal')

    plt.show()

def Plotter3D_single(data,elev=30,azim=-60):
    xvals,yvals,zvals = data
    #A plotter to see principal components in 3D
    plt.rc("xtick", labelsize=15)
    plt.rc("ytick", labelsize=15)

    z_min = zvals.min()
    z_max = zvals.max()
    z_normalized = (zvals - z_min) / (z_max - z_min)

    # Create figure for 3D plot
    fig = plt.figure(figsize=(8, 6), dpi=200)
    ax = fig.add_subplot(111, projection='3d')



    # Define the size of the bars
    dx = dy = 1.5  # Width of the bars in the x and y direction
    dz = z_normalized        # Height of the bars (z values)

    # Create 3D bar plot
    ax.bar3d(xvals, yvals, np.zeros_like(zvals), dx, dy, dz, color=plt.cm.plasma(z_normalized))

    ax.view_init(elev=elev, azim=azim) 

    # Setting labels and title
    ax.set_xlabel('Neutrons')
    ax.set_ylabel('Protons')
    ax.set_zlabel('Scaled Z')


    plt.show()    



def Plotter2D(ax,xvals,yvals,zvals):
    #A plotter to see principal components in 2D, part of the big plotter



    # Create scatter plot
    sc = ax.scatter(xvals, yvals, c=zvals, s=25, cmap='plasma',marker='s')
    # plt.colorbar(sc, label='Z-Value')
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label('Z-Value', fontsize=25) 
    cbar.ax.tick_params(labelsize=20) 




    plt.xlabel('Neutrons',fontsize=25)
    plt.ylabel('Protons',fontsize=25)

    ax.grid(True)
    ax.axis('equal')

    # plt.show()

def Plotter3D(ax,xvals,yvals,zvals,elev=30,azim=-60):
    #A plotter to see principal components in 3D, part of the big plotter


    z_min = zvals.min()
    z_max = zvals.max()
    z_normalized = (zvals - z_min) / (z_max - z_min)

    # Create figure for 3D plot
    # fig = plt.figure(figsize=(8, 6), dpi=200)
    # ax = fig.add_subplot(111, projection='3d')



    # Define the size of the bars
    dx = dy = 1.5  # Width of the bars in the x and y direction
    dz = z_normalized        # Height of the bars (z values)

    # Create 3D bar plot
    ax.bar3d(xvals, yvals, np.zeros_like(zvals), dx, dy, dz, color=plt.cm.plasma(z_normalized))

    ax.view_init(elev=elev, azim=azim) 

    # Setting labels and title
    ax.set_xlabel('Neutrons')
    ax.set_ylabel('Protons')
    ax.set_zlabel('Scaled Z')


    # plt.show()    



def PlotMultiple(data_sets,angles_sheet=None):
    # Determine the number of rows needed for the plots
    n_rows = len(data_sets)
    # Each data_sets element should look like [x,y,z]

    # Create a figure with subplots
    fig = plt.figure(figsize=(20, 8 * n_rows), dpi=200)

    for i, (xvals, yvals, zvals) in enumerate(data_sets):
        # 2D plot
        ax_2d = fig.add_subplot(n_rows, 2, 2 * i + 1)
        Plotter2D(ax_2d, xvals, yvals, zvals)

        # 3D plot
        ax_3d = fig.add_subplot(n_rows, 2, 2 * i + 2, projection='3d')
        if angles_sheet == None:
            Plotter3D(ax_3d, xvals, yvals, zvals)
        else:
            Plotter3D(ax_3d, xvals, yvals, zvals,elev=angles_sheet[0],azim=angles_sheet[1])

    plt.tight_layout()
    plt.show()



Next up are some helpful functions for splitting up datasets

In [ ]:
def separate_points_random(list1,random_chance):
    """
    Separates points in list1 into two groups randomly

    """
    train = []
    test = []

    train_list_coordinates=[]
    test_list_coordinates=[]


    for i in range(len(list1)):
        point1=list1[i]
        val=np.random.rand()
        if val<=random_chance:
            train.append(point1)
            train_list_coordinates.append(i)
        else:
            test.append(point1)
            test_list_coordinates.append(i)

    return np.array(train), np.array(test), np.array(train_list_coordinates), np.array(test_list_coordinates)


def separate_points_distance(list1, list2, distance):
    """
    Separates points in list1 into two groups based on their proximity to any point in list2.

    :param list1: List of (x, y) tuples.
    :param list2: List of (x, y) tuples.
    :param distance: The threshold distance to determine proximity.
    :return: Two lists - close_points and distant_points.
    """
    train = []
    test = []

    train_list_coordinates=[]
    test_list_coordinates=[]

    for i in range(len(list1)):
        point1=list1[i]
        close = False
        for point2 in list2:
            if np.linalg.norm(np.array(point1) - np.array(point2)) <= distance:
                close = True
                break
        if close:
            train.append(point1)
            train_list_coordinates.append(i)
        else:
            test.append(point1)
            test_list_coordinates.append(i)

    return np.array(train), np.array(test), np.array(train_list_coordinates), np.array(test_list_coordinates)

def separate_points_distance_allSets(list1, list2, distance1, distance2):
    """
    Separates points in list1 into three groups based on their proximity to any point in list2.

    :param list1: List of (x, y) tuples.
    :param list2: List of (x, y) tuples.
    :param distance: The threshold distance to determine proximity.
    :return: Two lists - close_points and distant_points.
    """
    train = []
    validation=[]
    test = []

    train_list_coordinates=[]
    validation_list_coordinates=[]
    test_list_coordinates=[]

    for i in range(len(list1)):
        point1=list1[i]
        close = False
        for point2 in list2:
            if np.linalg.norm(np.array(point1) - np.array(point2)) <= distance1:
                close = True
                break
        if close:
            train.append(point1)
            train_list_coordinates.append(i)
        else:
            close2=False
            for point2 in list2:
                if np.linalg.norm(np.array(point1) - np.array(point2)) <= distance2:
                    close2 = True
                    break
            if close2==True:
                validation.append(point1)
                validation_list_coordinates.append(i)
            else:
                test.append(point1)
                test_list_coordinates.append(i)                

    return np.array(train),np.array(validation), np.array(test), np.array(train_list_coordinates),  np.array(validation_list_coordinates),np.array(test_list_coordinates)


## Setting up the Models 

For this example application we are considering a simple nuclear model to generate the data so that we can have a firm control over truth that we will lack in the real case. In this case we are using an extended liquid drop model and variations on those models to construct a set of models ranging in 'badness'. These model definitions and model parameters are worth playing around with to check how the whole machinery works.

In [ ]:
def LDM_extended(params, x): 
    #x = (n,z)
    #params= parameters (volume, surface, curv, sym, ssym, sym_2, Coulomb)
    
    n=x[0]
    z=x[1]
    A = n + z
    I = (n-z)/(n+z)


    return A*params[0] + params[1] * A ** (2/3) + params[2] * A ** (1/3)  +  params[3] * I ** 2*A + \
                + params[4] * (I ** 2) * A ** (2/3) + params[5] * (I ** 4)*A + params[6]*((z**2)/((A)**(1/3))) 


### Model Parameters and Data Variables

The parameters themselves are changed in the following cells, along with several features controlling the noise added to the datasets and dataset splits.

In [ ]:
### Global variables####

# SkO values selected for the truth

truth_params = [-15.835, 17.3, 9, 31.98, -58, - 4.5 * (163.5 ** 2) * (0.1605**2) / 223.5,0.57]

PerfectM_params = [-15.835, 17.3, 9, 31.98, -58, - 4.5 * (163.5 ** 2) * (0.1605**2) / 223.5,0.57]

# SLy4 
GoodM_params = [-15.972, 18.4, 9, 32.01, -54, - 4.5 * (95.97 ** 2) * (0.1596**2) / 230.1, 0.57]
# NL_1
BadM_params = [-16.425, 18.8, 9, 43.48, -110, -4.5 *(311.18 **2) *(0.1518**2)/211.3, 0.57]

# NL_1 
TerribleM_params = [-15.972, 18.4, 9, 0, 0, 0, 0.57]

In [ ]:
#Noises
##########################

#Turn off term (to make colinear examples). Turn it to 1 or 0

switch=1

#Noise terms to be added to the parameters
params_noise_term_p=0.000*switch
params_noise_term_g=0.002*switch
params_noise_term_b=0.002*switch
params_noise_term_t=0.002*switch


overall_output_noise=1*switch   # Noise added to each observation coming from models

# overall_output_noise=0.000   # Noise added to each observation coming from models

overall_data_noise= 1*switch  #Noise added to the "true" generated data itself




#Data separation (training, testing, validation) set ups
##########################

#Number used for when dividing training and testing randomly uniformly (interpolating)
TestingFraction=0.34

#Number used for when dividing training and testing by how close they are to the stable nuclei
distance=2

#Coordinates to truncate so the LDM evaluations are not done below these isotopes
minimumZ=8
minimumN=8

#Even-even nuclei only? Set this flag to True
even_even=True

#Centering set up
##########################

centering_data=True


#Scenario set up
##########################

n_perfect =0
n_good = 3
n_bad = 5
n_terrible=10

n_classes=[n_perfect,n_good,n_bad,n_terrible]

#Re-labeling terrible by "bad" so we are not too insensitive to the poor model
n_Labels=["Perfect", "Good", "Inter.","Bad"]



#Isotope chain for 1-D plotting selection
##########################

Selected_element=50
Selected_element_name="Sn"



#Computing constrained MCMC? (takes ~ + 5 minutes extra time)
##########################

computing_MCMC=True

### Dataset Preparation

Now we should chop up our dataset and select our training, testing, and validation sets. We will use the stable nuclei as a starting point and then select everything else based around those

In [ ]:
stable_coordinates_full=np.loadtxt("Stable-Isotopes.txt")

stable_coordinates=[]

if(even_even):
    for i in range(len(stable_coordinates_full)):
        if (stable_coordinates_full[i][0]>=minimumN) & (stable_coordinates_full[i][1]>=minimumZ) & (stable_coordinates_full[i][0]% 2 == 0) & (stable_coordinates_full[i][1]% 2 == 0) :
            stable_coordinates.append(stable_coordinates_full[i])
else:
    for i in range(len(stable_coordinates_full)):
        if (stable_coordinates_full[i][0]>=minimumN) & (stable_coordinates_full[i][1]>=minimumZ) :
            stable_coordinates.append(stable_coordinates_full[i])

stable_coordinates=np.array(stable_coordinates)


Full_set_2020=np.loadtxt("AME2020.txt")
AME2020_even=[]

if(even_even):
    for i in range(len(Full_set_2020)):
        if (Full_set_2020[i][0]>=minimumN) & (Full_set_2020[i][1]>=minimumZ) & (Full_set_2020[i][0]% 2 == 0) & (Full_set_2020[i][1]% 2 == 0) :
            AME2020_even.append([Full_set_2020[i][0],Full_set_2020[i][1]])
else:
    for i in range(len(Full_set_2020)):
        if (Full_set_2020[i][0]>=minimumN) & (Full_set_2020[i][1]>=minimumZ) :
            AME2020_even.append([Full_set_2020[i][0],Full_set_2020[i][1]])

Full_set=np.array(AME2020_even)

distance1=2
distance2=3

training_set, validation_set, test_set,train_coordinates0, validation_coordinates0,test_coordinates0=separate_points_distance_allSets(Full_set, stable_coordinates, distance1,distance2)

train_coordinates=[]
for i in range(len(Full_set)):
    isotope2020=Full_set[i]
    for j in range(len(training_set)):
       train_isotope=training_set[j]
       if (isotope2020[0]==train_isotope[0]) & (isotope2020[1]==train_isotope[1]):
           train_coordinates.append(i)
           break
train_coordinates=np.array(train_coordinates)

validation_coordinates=[]
for i in range(len(Full_set)):
    isotope2020=Full_set[i]
    for j in range(len(validation_set)):
       validation_isotope=validation_set[j]
       if (isotope2020[0]==validation_isotope[0]) & (isotope2020[1]==validation_isotope[1]):
           validation_coordinates.append(i)
           break
validation_coordinates=np.array(validation_coordinates)

test_coordinates=[]
for i in range(len(Full_set)):
    isotope2020=Full_set[i]
    for j in range(len(test_set)):
       test_isotope=test_set[j]
       if (isotope2020[0]==test_isotope[0]) & (isotope2020[1]==test_isotope[1]):
           test_coordinates.append(i)
           break
test_coordinates=np.array(test_coordinates)

### Plotting the data landscape

It can be instructive to visual your data split, especially if you're trying to construct a super model that is tuned for extrapolation

In [ ]:
plt.rc("xtick", labelsize=35)
plt.rc("ytick", labelsize=35)

fig, ax = plt.subplots(figsize=(12,10), dpi=100)


color_trainig=colors[9]
color_validation='orange'
color_testing=colors[3]

marker_trainig='s'
marker_validation='*'
marker_testing='o'



size_trainig=30
size_validation=80
size_testing=35

alpha_trainig=0.8
alpha_validation=0.9
alpha_testing=0.4



ax.scatter(x = training_set.T[0], y = training_set.T[1], label = "Train ("+ str(len(training_set))+")", alpha = alpha_trainig,color=color_trainig,s=size_trainig,marker=marker_trainig)

ax.scatter(x = validation_set.T[0], y = validation_set.T[1], label = "Valid. (" + str(len(validation_set))+")", alpha = alpha_validation,color=color_validation,s=size_validation,marker=marker_validation)

ax.scatter(x = test_set.T[0], y = test_set.T[1], label = "Test (" + str(len(test_set))+")", alpha = alpha_testing,color=color_testing,s=size_testing,marker=marker_testing)


ax.scatter(x = stable_coordinates.T[0], y = stable_coordinates.T[1], label = "Stable", alpha = 0.8,color='black',s=11,marker="s")



plt.xlabel('')

plt.annotate('Neutrons', xy=(0.35, 0.1), xycoords='axes fraction',
             ha='center', va='top', fontsize=45) 
plt.ylabel('')

plt.annotate('Protons', xy=(0.05,0.7), xycoords='axes fraction',
             ha='center', va='top', fontsize=45,rotation =90) 


# plt.ylabel("Protons",fontsize=45)
plt.legend(fontsize=34,markerscale=3 ,loc =(0.53,0.03))
plt.show()

### Constructing the different models

While we've defined the base for our perfect, good, intermediate, and bad models above, we now consider the situation where slight (statistical, in this case) variations of these models are introduced as new models

In [ ]:


# masses_truth = LDM_extended(truth_params,[input_NZ["Z"], input_NZ["N"]]) 

models_output = {}
models_output_train = {}
models_output_validation = {}
models_output_test = {}





# #Option 1: Constructing the data by just adding a random noise to the output of the models
# #Perfect Models Loop
# params = PerfectM_params
# for i in range(n_perfect):
#     models_output[str("PerfectModel_")+str(i)] = LDM_extended(params,[input_NZ["Z"], input_NZ["N"]])+ np.random.normal(0,1,size=len(input_NZ)) * noise_term_p


# #Good Models Loop
# params = GoodM_params
# for i in range(n_good):
#     models_output[str("GoodModel_")+str(i)] = LDM_extended(params,[input_NZ["Z"], input_NZ["N"]])+ np.random.normal(0,1,size=len(input_NZ)) * noise_term_g

# #Bad Models Loop
# params = BadM_params
# for i in range(n_bad):
#     models_output[str("IntermediateModel_")+str(i)] = LDM_extended(params,[input_NZ["Z"], input_NZ["N"]])+ np.random.normal(0,1,size=len(input_NZ)) * noise_term_b

# #Terrible Models Loop
# params = TerribleM_params
# for i in range(n_terrible):
#     models_output[str("BadModel_")+str(i)] = LDM_extended(params,[input_NZ["Z"], input_NZ["N"]])+ np.random.normal(0,1,size=len(input_NZ)) * noise_term_t




#Option 2: Constructing the data by adding a random noise to the parameters of the models, plus random noise to the output
#Perfect Models Loop
np.random.seed(142857)
params = PerfectM_params
for i in range(n_perfect):
    ran_params=params + np.random.normal(0,1,size=len(params))*params*params_noise_term_p
    models_output[str("PerfectModel_")+str(i)] = LDM_extended(ran_params,Full_set.T)+np.random.normal(0,1,size=len(Full_set)) * overall_output_noise



#Good Models Loop
np.random.seed(542857)
params = GoodM_params
for i in range(n_good):

    ran_params=params+ np.random.normal(0,1,size=len(params))*params*params_noise_term_g


    models_output[str("GoodModel_")+str(i)] = LDM_extended(ran_params
                                                           ,Full_set.T)+ np.random.normal(0,1,size=len(Full_set)) * overall_output_noise




#Bad Models Loop
np.random.seed(342857)
params = BadM_params
for i in range(n_bad):
    ran_params=params+ np.random.normal(0,1,size=len(params))*params*params_noise_term_b
    models_output[str("IntermediateModel_")+str(i)] = LDM_extended(ran_params,Full_set.T)+ np.random.normal(0,1,size=len(Full_set)) * overall_output_noise


    



#Terrible Models Loop
np.random.seed(442857)
params = TerribleM_params
for i in range(n_terrible):
    ran_params=params+ np.random.normal(0,1,size=len(params))*params*params_noise_term_t


    models_output[str("BadModel_")+str(i)] = LDM_extended(ran_params
                                                          ,Full_set.T)+ np.random.normal(0,1,size=len(Full_set)) * overall_output_noise
    


#This is done here to have a list of pure models, without the ground truth
key_list=list(models_output.keys())
np.random.seed(7*142857)
models_output['truth']=LDM_extended(truth_params,Full_set.T) + np.random.normal(0,1,size=len(Full_set))*overall_data_noise


models_output = pd.DataFrame(models_output)
models_output["N"] = Full_set.T[0]
models_output["Z"] = Full_set.T[1]
models_output["A"] = models_output["N"] + models_output["Z"]



#Separating outputs for the three regions
models_output_train = models_output.iloc[train_coordinates]

models_output_validation = models_output.iloc[validation_coordinates]

models_output_test = models_output.iloc[test_coordinates]


Now we can plot those models to see how bad they truly are when compared to 'truth'

In [ ]:
plt.rc("xtick", labelsize=25)
plt.rc("ytick", labelsize=25)

fig, ax = plt.subplots(figsize=(12,10), dpi=100)


model_index=0
class_index=0
for n_vals in n_classes:
    legend_flag=0
    for i in range(n_vals):
        if legend_flag==0:
            legend_flag=1
            ax.scatter(x = models_output["A"], y = models_output[key_list[model_index]]/models_output["A"], label = n_Labels[class_index], alpha = 0.4,color=colors[class_index],marker=markers[0],s=60)
        else:
            ax.scatter(x = models_output["A"], y = models_output[key_list[model_index]]/models_output["A"],  alpha = 0.3,color=colors[class_index],marker=markers[0],s=50)
        model_index=model_index+1
    class_index=class_index+1
    legend_flag=0





ax.scatter(x = models_output["A"], y = models_output['truth']/models_output["A"], label = "Truth", alpha = 0.7,color='k',s=20)


plt.xlabel("A",fontsize=35)
plt.ylabel("BE/A MeV",fontsize=35)
plt.legend(fontsize=25,markerscale=2 )

plt.show()

In [ ]:
plt.rc("xtick", labelsize=25)
plt.rc("ytick", labelsize=25)

fig, ax = plt.subplots(figsize=(12,10), dpi=100)


model_index=0
class_index=0
for n_vals in n_classes:
    legend_flag=0
    for i in range(n_vals):
        if legend_flag==0:
            legend_flag=1
            ax.scatter(x = models_output["A"], y = models_output[key_list[model_index]], label = n_Labels[class_index], alpha = 0.4,color=colors[class_index],marker=markers[0],s=60)
        else:
            ax.scatter(x = models_output["A"], y = models_output[key_list[model_index]],  alpha = 0.3,color=colors[class_index],marker=markers[0],s=50)
        model_index=model_index+1
    class_index=class_index+1
    legend_flag=0





ax.scatter(x = models_output["A"], y = models_output['truth'], label = "Truth", alpha = 0.7,color='k',s=20)


plt.xlabel("A",fontsize=35)
plt.ylabel("BE MeV",fontsize=35)
plt.legend(fontsize=25,markerscale=2 )

plt.show()